In [1]:
import pandas as pd

In [2]:
from nltk.tokenize import sent_tokenize


In [3]:
from bs4 import BeautifulSoup


In [20]:
def remove_tags(html):
 
    # parse html content
    soup = BeautifulSoup(html, "html.parser")
 
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
 
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)
 
 


In [18]:
f = open("sociology.html", "r", encoding="utf-8")
text=f.read()
f.close()

In [23]:
text=remove_tags(text)
soc = filt(sent_tokenize(text))

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lucia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
def filt(sents):
    ret=[]
    for s in sents:
        if len(s)>45:
            ret.append(s)
    return ret

In [6]:
f = open("desc.txt", "r", encoding="utf-8")
text=f.read()
f.close()
text=text.replace('\r', ' ').replace('\n', ' ')
text=text.split("PREFACE TO THE SECOND EDITION.")[2]
text=text.split("probability to the action of sexual selection.")[0]
text+="probability to the action of sexual selection."
text=text.strip()
dom = filt(sent_tokenize(text))

In [51]:
f = open("humboldt.txt", "r", encoding="utf-8")
text=f.read()
f.close()
text=text.replace("\n"," ")
text=text.split("EDITOR'S PREFACE.")[2]
text=text.strip()
mm = sent_tokenize(text)

In [38]:
f = open("gutenberg.org_cache_epub_145_pg145.txt", "r", encoding="utf-8")
text=f.read()
f.close()
text=text.replace("\n"," ")
text=text.split("PRELUDE.")[2]
text=text.strip()
mm = sent_tokenize(text)

In [27]:
f = open("gutenberg.org_files_7116_7116-0.txt", "r", encoding="utf-8")
text=f.read()
f.close()
text=text.replace("\n"," ")
text=text.split("THEIR ARRANGEMENTS AND FORMATION.")[1]
text=text.strip()
vest = sent_tokenize(text)


In [38]:
soc[1]

'The first kind, Inorganic Evolution, which, had it been dealt with, would have occupied two volumes, one dealing with Astrogeny and the other with Geogeny, was passed over because it seemed undesirable to postpone the more important applications of the doctrine for the purpose of elaborating those less important applications which logically precede them.'

In [12]:
f = open("oos.txt", "r", encoding="utf-8")
text=f.read()
f.close()
text=text.replace("\n"," ")
text=text.split("When on board H.M.S. ‘Beagle,’ as naturalist,")[1]
text=text.split("from so simple a beginning endless forms most beautiful and most wonderful have been, and are being, evolved.")[0]
text+="from so simple a beginning endless forms most beautiful and most wonderful have been, and are being, evolved."
text=text.strip()
oos = filt(sent_tokenize(text))


In [9]:
f = open("spen.txt", "r", encoding="utf-8")
text=f.read()
f.close()
text=text.replace("\n"," ")
text=text.split("TO THE REVISED AND ENLARGED EDITION.")[1]

text=text.split("he was not unwarranted in representing my conception in the way that he has done.")[0]

text=text.strip()

spen=filt(sent_tokenize(text))

In [10]:
spen[-1]

'At the same time I willingly admit that, in the absence of certain statements which I have now supplied,'

In [11]:
import glob

In [251]:
#path=glob.glob("biology/methodsstudyinn03agasgoog/*_djvu.txt")[0]
path="muller.txt"

In [252]:
f = open(path, "r", encoding="utf-8")
text=f.read()
f.close()
text=text.replace("\n"," ")
#text=text.split("TO THE REVISED AND ENLARGED EDITION.")[1]
text=text.strip()

spider=sent_tokenize(text)

In [11]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

In [12]:

#model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
model = SentenceTransformer('thenlper/gte-small')

In [26]:
emb_dom = model.encode(dom)
emb_soc=model.encode(soc)

In [27]:
emb_oos = model.encode(oos)


In [13]:
emb_spen = model.encode(spen)


In [52]:
emb_vest=model.encode(mm)

In [253]:
spi=model.encode(spider)

In [15]:
import pickle


In [30]:
with open('vec_dom.pickle', 'wb') as handle:
    pickle.dump(emb_dom, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
with open('vec_oos.pickle', 'wb') as handle:
    pickle.dump(emb_oos, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [16]:

with open('vec_spen.pickle', 'wb') as handle:
    pickle.dump(emb_spen, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [33]:

with open('vec_soc.pickle', 'wb') as handle:
    pickle.dump(emb_soc, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [22]:
from scipy.spatial.distance import cdist
import numpy as np

In [36]:
emb_oos.shape

NameError: name 'emb_oos' is not defined

In [23]:
file = open('vec_oos.pickle', 'rb')
emb_oos = pickle.load(file)
file.close()
file = open('vec_dom.pickle', 'rb')
emb_spen = pickle.load(file)
file.close()

In [45]:
emb_oos=np.array(emb_oos)

In [46]:
emb_spen=np.array(emb_spen)
#emb_vest=np.array(emb_vest)

In [27]:
emb_muller=np.array(spi)

NameError: name 'spi' is not defined

In [47]:
dists2=cdist(emb_spen, emb_oos, "cosine")


In [72]:
np.histogram(dists2)

(array([      21,      361,     7545,   186415,  2572750, 15092281,
        25529818,  7832790,   260743,      540], dtype=int64),
 array([0.00562693, 0.04046429, 0.07530165, 0.11013901, 0.14497637,
        0.17981373, 0.21465109, 0.24948846, 0.28432582, 0.31916318,
        0.35400054]))

In [67]:
matches=np.asarray((dists2>0.1)&(dists2<0.11)).nonzero()
#matches=np.asarray(matches<0.2).nonzero()
#matches=np.asarray((dists2<0.05)).nonzero()


In [68]:
len(matches[0])

4894

In [69]:
for i in range(0,len(matches[0])):
    
    i1=matches[0][i]
    i2=matches[1][i]
    
    

    print()
    print(dom[i1])
    print(oos[i2])


I may take this opportunity of remarking that my critics frequently assume that I attribute all changes of corporeal structure and mental power exclusively to the natural selection of such variations as are often called spontaneous; whereas, even in the first edition of the ‘Origin of Species,’ I distinctly stated that great weight must be attributed to the inherited effects of use and disuse, with respect both to the body and mind.
We must by no means overlook the effects of the definite action of changed conditions of life, of so-called spontaneous variations, which seem to depend in a quite subordinate degree on the nature of the conditions, of the tendency to reversion to long-lost characters, of the complex laws of growth, such as of correlation, comprehension, of the pressure of one part on another, &c., and finally of sexual selection, by which characters of use to one sex are often gained and then transmitted more or less perfectly to the other sex, though of no use to the sex